# Study Designer Example (DNA sequencing)

In [ ]:
from ipywidgets import (RadioButtons, VBox, HBox, Layout, Label, Checkbox, Text, IntSlider)
from qgrid import show_grid
label_layout = Layout(width='100%')
from isatools.create.models import *
from isatools.model import Investigation
from isatools.isatab import dump_tables_to_dataframes as dumpdf
import qgrid
qgrid.nbinstall(overwrite=True)

## Sample planning section

### Study design type

Please specify if the study is an intervention or an observation.

In [ ]:
rad_study_design = RadioButtons(options=['Intervention', 'Observation'], value='Intervention', disabled=False)
VBox([Label('Study design type?', layout=label_layout), rad_study_design])

### Intervention study

If specifying an intervention study, please answer the following:
 - Are study subjects exposed to a single intervention or to multiple intervention?
 - Are there 'hard to change' factors, which restrict randomization of experimental unit?
 
*Note: if you chose 'observation' as the study design type, the following choices will be disabled and you should skip to the Observation study section*
 

In [ ]:
if rad_study_design.value == 'Intervention':
    study_design = InterventionStudyDesign()
if rad_study_design.value == 'Observation':
    study_design = None
intervention_ui_disabled = not isinstance(study_design, InterventionStudyDesign)
intervention_type = RadioButtons(options=['single', 'multiple'], value='single', disabled=intervention_ui_disabled)
intervention_type_vbox = VBox([Label('Single intervention or to multiple intervention?', layout=label_layout), intervention_type])
free_or_restricted_design = RadioButtons(options=['yes', 'no'], value='no', disabled=intervention_ui_disabled)
free_or_restricted_design_vbox = VBox([Label("Are there 'hard to change' factors?", layout=label_layout), free_or_restricted_design])
HBox([intervention_type_vbox, free_or_restricted_design_vbox])

In [ ]:
hard_to_change_factors_ui_disabled = free_or_restricted_design.value == 'no'
hard_to_change_factors = RadioButtons(options=[1, 2], value=1, disabled=hard_to_change_factors_ui_disabled)
VBox([Label("If applicable, how many 'hard to change factors'?", layout=label_layout), hard_to_change_factors])

In [ ]:
repeats = intervention_type.value != 'single'
factorial_design = free_or_restricted_design.value == 'no'
split_plot_design = (free_or_restricted_design.value == 'yes' and hard_to_change_factors.value == 1)
split_split_plot_design = (free_or_restricted_design.value == 'yes' and hard_to_change_factors.value == 2)
print('Interventions: {}'.format('Multiple interventions' if repeats else 'Single intervention'))
design_type = 'factorial'  # always default to factorial
if split_plot_design:
    design_type = 'split plot'
elif split_split_plot_design:
    design_type = 'split split'
print('Design type: {}'.format(design_type))

#### Factorial design - intervention types

If specifying an factorial design, please list the intervention types here.

In [ ]:
factorial_design_ui_disabled = not factorial_design
chemical_intervention = Checkbox(value=True, description='Chemical intervention', disabled=factorial_design_ui_disabled)
behavioural_intervention = Checkbox(value=False, description='Behavioural intervention', disabled=factorial_design_ui_disabled)
surgical_intervention = Checkbox(value=False, description='Surgical intervention', disabled=factorial_design_ui_disabled)
biological_intervention = Checkbox(value=False, description='Biological intervention', disabled=factorial_design_ui_disabled)
radiological_intervention = Checkbox(value=False, description='Radiological intervention', disabled=factorial_design_ui_disabled)
VBox([chemical_intervention, surgical_intervention, biological_intervention, radiological_intervention])

In [ ]:
level_uis = []
if chemical_intervention:
    agent_levels = Text(
        value='calpol, no agent',
        placeholder='e.g. cocaine, calpol',
        description='Agent:',
        disabled=False
    )
    dose_levels = Text(
        value='low, high',
        placeholder='e.g. low, high',
        description='Dose levels:',
        disabled=False
    )
    duration_of_exposure_levels = Text(
        value='short, long',
        placeholder='e.g. short, long',
        description='Duration of exposure:',
        disabled=False
    )
VBox([Label("Chemical intervention factor levels:", layout=label_layout), agent_levels, dose_levels, duration_of_exposure_levels])

In [ ]:
factory = TreatmentFactory(intervention_type=INTERVENTIONS['CHEMICAL'], factors=BASE_FACTORS)
for agent_level in agent_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[0], agent_level.strip())
for dose_level in dose_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[1], dose_level.strip())
for duration_of_exposure_level in duration_of_exposure_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[2], duration_of_exposure_level.strip())
print('Number of study groups (treatment groups): {}'.format(len(factory.compute_full_factorial_design())))
treatment_sequence = TreatmentSequence(ranked_treatments=factory.compute_full_factorial_design())

Next, specify if all study groups of the same size, i.e have the same number of subjects? (in other words, are the groups balanced).

In [ ]:
group_blanced = RadioButtons(options=['Balanced', 'Unbalanced'], value='Balanced', disabled=False)
VBox([Label('Are study groups balanced?', layout=label_layout), group_blanced])

Provide the number of subject per study group:

In [ ]:
group_size = IntSlider(value=5, min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size

In [ ]:
plan = SampleAssayPlan(group_size=group_size.value)

In [ ]:
rad_sample_type = RadioButtons(options=['Blood', 'Sweat', 'Tears', 'Urine'], value='Blood', disabled=False)
VBox([Label('Sample type?', layout=label_layout), rad_sample_type])

How many times each of the samples have been collected?

In [ ]:
sampling_size = IntSlider(value=3, min=0, max=100, step=1, description='Sample size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
sampling_size

In [ ]:
plan.add_sample_type(rad_sample_type.value)
plan.add_sample_plan_record(rad_sample_type.value, sampling_size.value)
isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)

## Generate ISA model objects from the sample plan and render the study-sample table

In [ ]:
isa_investigation = Investigation(identifier='inv101')
isa_study = isa_object_factory.create_study_from_plan()
isa_study.filename = 's_study.txt'
isa_investigation.studies = [isa_study]
dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

In [ ]:
print('Total rows generated: {}'.format(len(sample_table)))

## Assay planning 

### Select assay technology type to map to sample type from sample plan

In [ ]:
rad_assay_type = RadioButtons(options=['DNA microarray', 'DNA sequencing', 'Mass spectrometry', 'NMR spectroscopy'], value='DNA sequencing', disabled=False)
VBox([Label('Assay type to map to sample type "{}"?'.format(rad_sample_type.value), layout=label_layout), rad_assay_type])

In [ ]:
if rad_assay_type.value == 'DNA sequencing':
    assay_type = AssayType(measurement_type='genome sequencing', technology_type='nucleotide sequencing')
    print('Selected measurement type "genome sequencing" and technology type "nucleotide sequencing"')
else:
    raise Exception('Assay type not implemented')

### Topology modifications

In [ ]:
technical_replicates = IntSlider(value=2, min=0, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
technical_replicates

In [ ]:
inst_mod_454gs = Checkbox(value=True, description='Instrument: 454 GS')
inst_mod_454gsflx = Checkbox(value=True, description='Instrument: 454 GS FLX')
VBox([inst_mod_454gs, inst_mod_454gsflx])

In [ ]:
instruments = set()
if inst_mod_454gs.value: instruments.add('454 GS')
if inst_mod_454gsflx.value: instruments.add('454 GS FLX')
top_mods = AssayTopologyModifiers(technical_replicates=technical_replicates.value, instruments=instruments)
print('Technical replicates: {}'.format(top_mods.technical_replicates))
assay_type.topology_modifiers = top_mods
plan.add_assay_type(assay_type)
plan.add_assay_plan_record(rad_sample_type.value, assay_type)
assay_plan = next(iter(plan.assay_plan))
print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
if len(top_mods.instruments) > 0:
    print('Instruments: {}'.format(list(top_mods.instruments)))

## Generate ISA model objects from the assay plan and render the assay table

In [ ]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]
for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, len(assay.samples), len(assay.process_sequence), len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

In [ ]:
show_grid(dataframes[next(iter(dataframes.keys()))])